In [2]:
# pip install py2neo
# pip install neo4j
#from py2neo import Graph
from neo4j import GraphDatabase
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns
import numpy as np
from graph_model import 

    100% |████████████████████████████████| 184kB 32.7MB/s ta 0:00:01
  Running setup.py bdist_wheel for neo4j ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/c0/dc/2e/6a26c3a5c63bb49a81092caa5ee48895971c0f229ee7900e7d
  Running setup.py bdist_wheel for neobolt ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/42/54/00/270de525f8f966d864bd3a0224d38dca2573b53c1a54267720
  Running setup.py bdist_wheel for neotime ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/d4/b4/c4/ab867cd0ac9e667f8cd269f9364fd1cb8ee75fe50f5ce97c79
Successfully built neo4j neobolt neotime
You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


This notebook goes through converting some of the features of our data into more easily usable structures in the graph model

In [5]:
graph = Graph("bolt://3.220.233.169:7687", auth=("neo4j", "i-0e23d19f0d8795714"))


This cypher command returns the current constraints and indices that are set in the database, and then shows the the types of relationships that exist between types of nodes in the graph. It is one of the ways I use to make sure that queries are being sent and recceived correctly from the database, and make sure the database itself ihas not been messed up in some way.


In [29]:
graph.run("CALL db.schema()").data()


[{'nodes': [(_-22:Category {constraints: [], indexes: ['name'], name: 'Category'}),
   (_-23:User {constraints: ['CONSTRAINT ON ( user:User ) ASSERT user.id IS UNIQUE'], indexes: [], name: 'User'}),
   (_-24:Review {constraints: ['CONSTRAINT ON ( review:Review ) ASSERT review.id IS UNIQUE'], indexes: [], name: 'Review'}),
   (_-25:Reputation {constraints: [], indexes: [], name: 'Reputation'}),
   (_-21:Business {constraints: ['CONSTRAINT ON ( business:Business ) ASSERT business.id IS UNIQUE'], indexes: [], name: 'Business'})],
  'relationships': [(Review)-[:REVIEWS {}]->(Business),
   (User)-[:WROTE {}]->(Review),
   (User)-[:FRIEND {}]->(User),
   (User)-[:COOL_LEVEL {}]->(Reputation),
   (User)-[:FUNNY_LEVEL {}]->(Reputation),
   (Business)-[:IN_CATEGORY {}]->(Category)]}]

## User Feature Nodes ##

These are distributions of funny, cool and useful responses that I want to divide into bins based roughly on quantiles. I will then create nodes in the graph for each bin and connect the user nodes to the bin they are put in. This makes it much more efficient and easy to query based on these bins, which will be a  part of the graph based reccomendation algorithm. As part of this process I added the nodes for the cool distribution and connected them individually on the server itself to make sure I was adding things the way I intended. I then made the loops below to automate some of the process. Adding the cool nodes and edges with loops would only require some modifications of the code below.

| qunatile | ident | u.cool | u.funny | u.useful |
| --- | --- | --- | --- | --- |
| 0.20 | 1072521.40	| 0.0 | 0.0 | 0.0 |
| 0.35 | 1968091.95 | 0.0	 |0.0	|1.0 |
| 0.50 |	3063662.50	| 0.0	 | 0.0	| 3.0 | 
| 0.65 |	4389233.05	| 1.0	 | 1.0	| 7.0 |
| 0.80 |	5984803.60	| 5.0	 | 4.0	| 19.0 |
| 0.95 |	7830374.15	| 36.0	| 33.0	| 110.0 |


As a note some of the queries below are sent using py2neo, and then later I switched to using the standard neo4j driver with functions I wrote, so the syntax and packages requried will be different in some cases.

### Funny Edges ###

In [24]:
#the 0 group needs be done seperately because of the way cypher handles nodes that do not have a funny value.
graph.run("""
MATCH (n)
WHERE ID(n)= 6543
MATCH (u:User) WHERE NOT exists(u.funny)
CREATE (u)-[:FUNNY_LEVEL]->(n)
""")

In [ ]:
#this is for testing the funny edge adding loop
id_list=[(6543,0,1),(6668,1,2),(6552,2,33),(6561,33,999999)]
dflist=[]
for index in id_list:
    tempdf=graph.run(f"\
    MATCH (n)\
    WHERE ID(n)={index[0]}\
    MATCH (u:User) WHERE {index[1]}=<u.funny<{index[2]}\
    RETURN n.funny, u.name, u.funny LIMIT 6").to_data_frame()
    dflist.append(tempdf)

In [25]:
this is for adding the funny edges
id_list=[(6668,1,2),(6552,2,33),(6561,33,999999)]
dflist=[]


for index in id_list:
    tempdf=graph.run(f"\
    MATCH (n)\
    WHERE ID(n)={index[0]}\
    MATCH (u:User) WHERE {index[1]}<=u.funny<{index[2]}\
    CREATE (u)-[:FUNNY_LEVEL]->(n)").to_data_frame()
    dflist.append(tempdf)


### Useful Edges ###

In [34]:
#the 0 group needs be done seperately because of the way cypher handles nodes that do not have a useful value.
graph.run("""
MATCH (n)
WHERE ID(n)= 6670
MATCH (u:User) WHERE NOT exists(u.useful)
CREATE (u)-[:USEFUL_LEVEL]->(n)
""")

In [32]:
#this is for testing the useful edge adding loop
id_list=[(6674,0,1),(6680,1,3),(6681,3,7),(6682,7,19),(6683,110,999999)]
dflist=[]
for index in id_list:
    tempdf=graph.run(f"\
    MATCH (n)\
    WHERE ID(n)={index[0]}\
    MATCH (u:User) WHERE {index[1]}<u.useful<={index[2]}\
    RETURN n.useful, u.name, u.useful LIMIT 6").to_data_frame()
    dflist.append(tempdf)
    

In [38]:
#this is for adding the useful edges
id_list=[(6674,0,1),(6680,1,3),(6681,3,7),(6682,7,19),(?????,19,110),(6683,110,999999)]
dflist=[]

for index in id_list:
    tempdf=graph.run(f"\
    MATCH (n)\
    WHERE ID(n)={index[0]}\
    MATCH (u:User) WHERE {index[1]}<u.useful<={index[2]}\
    CREATE (u)-[:USEFUL_LEVEL]->(n)").to_data_frame()
    dflist.append(tempdf)


## Business Feature Nodes ##


The Business nodes had category feature nodes and in_category edges created as the data was being imported from the json files (see data_import.txt). I still wanted to be able to quickly subset data based on geographical location for testing purposes. This was important because taking random samples of data to run tests on can lead to very sparse data, whereas making sure samples come from the same geographical area helps make sure we will have enough related data to run meaningful analysis.

In [67]:
#this gets us a list of all the cities listed for each business.
with driver.session() as session:
    result = session.run("\
MATCH (b:Business) \
RETURN distinct b.city")

cities=result.values()

cities_df = pd.DataFrame(cities, columns = ['city'])

cities_df.shape

In [73]:
# this is to remove extraneous white space and lower case everything to consolidate city names with typos.
cities_df2['city']=cities_df['city'].str.strip()
cities_df2['city']=cities_df['city'].str.lower()

cities_df2=cities_df2.drop_duplicates()

cities_df2.shape

In [74]:
cities_df2.head()

,city
0,phoenix
1,mississauga
2,charlotte
3,goodyear
4,calgary


In [92]:
#this loop creates the city nodes
for city in cities_df2['city']:
    with driver.session() as session:
        result = session.run(f'\
    CREATE (c:City) \
    SET c.name="{city}" ')

In [ ]:
# this loop adds the IN_CITY edges to to each business
with driver.session() as session:
    result = session.run('\
    MATCH (b:Business) \
    MATCH (c:City) WHERE c.name=tolower(trim(b.city))\
    SET c += {state:b.state}\
    CREATE (b)-[:IN_CITY]->(c)')

Now that the city nodes and edges are in the graph I wanted to make sure each business has exactly one IN_CITY edge. I had to go in manually and fix two of the edges.

In [127]:
with driver.session() as session:
    result = session.run('\
    MATCH (b:Business) \
    RETURN apoc.node.degree(b,"IN_CITY")')

bizdegree=result.values()


bizdegdf=pd.DataFrame(bizdegree)

bizdegdf[0].value_counts()

Now that I had the cypher queries worked out I was able query and add the states in one loop

In [148]:
with driver.session() as session:
    result = session.run('\
    MATCH (c:City) \
    MERGE (s:State {name:c.state}) \
    ON CREATE SET s.name=c.state \
    MERGE (c)-[:IN_STATE]->(s)')

In [ ]:
# I used a similar loop from the cities to make sure that each city had only one state
with driver.session() as session:
    result = session.run('\
    MATCH (s:State)<-[:IN_STATE]-(c:City)\
    RETURN s.name, apoc.node.degree(s,"IN_STATE"), c.name, apoc.node.degree(c,"IN_CITY")')

counts_by_state=result.values()

countsdf=pd.DataFrame(counts_by_state)

countsdf.groupby(0).sum()

Later I came back to add edges between users and the states they have reviewed businesses in, so it uses the cypher function from graph_model.py

In [ ]:
cypher(driver,"MATCH (u:User)-[:WROTE]->(:Review)-[:REVIEWS]->(:Business)-[:IN_CITY]->(:City)-[:IN_STATE]->(s:State)\
CREATE (u)-[:REVIEWS_IN]->(s)", None)